<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ec53de2ce8711ed62b0bc5c69d95bc404c29b94f35e10a2cc1121b85dca92a8f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 13:11:39
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: 37.39 K (0.27%)
Current PnL: -28.92 L (-18.68%)
CY Booked + Current PnL: -14.63 L (-9.45%)
-------------------
Total profit:  1.24 L
Total loss:  -30.16 L
-------------------
Total Booked + Current PnL: 12.35 L (9.66%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.55 L (67.13%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.87,-16.68,20.12,0.09,16895.0,-16806.0,83971.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.32,OX40N,NTT,DURABLES
50,MASFIN,398.61,0.13,-2.10,24.66,22.05,23655.0,-2055.0,95925.0,-15.54,56.0,H-SC,7.10,164.0,-0.09,0.71,39.43,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.19,-13.90,16.17,0.02,24302.0,-24271.0,150288.0,129.35,48.0,M-SC,16.14,234.0,-1.00,1.12,30.72,OX40N,NTT,PAINTS
43,ITC,452.00,-0.33,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,45.0,X-LC,1.76,5.0,-0.15,1.46,3.63,X40,NTT,FMCG
82,VOLTAS,1530.00,0.51,5.43,13.53,19.69,27352.0,10413.0,202155.0,-4.81,47.0,X-MC,2.56,78.0,0.38,1.50,12.60,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SONACOMS,806.63,3.98,-15.18,64.52,39.54,55359.0,-15357.0,85802.0,-33.70,50.0,M-SC,4.63,220.0,-0.28,0.64,20.99,AR,ATH,AUTO
22,DIXON,18931.72,3.52,-17.89,48.07,21.58,55316.0,-25065.0,115074.0,-58.62,26.0,X-MC,7.13,56.0,-0.45,0.86,3.52,X40N,ATH,IT
28,GREENPANEL,537.00,3.38,-36.43,125.18,43.14,145682.0,-66700.0,116378.0,160.98,35.0,M-SC,9.47,240.0,-0.46,0.87,9.47,XY24,NTT,MISC
18,COFFEEDAY,80.00,3.26,-43.19,138.10,35.27,89091.0,-49037.0,64512.0,-55.73,36.0,L-SC,10.74,270.0,-0.55,0.48,57.89,XR,NTT,HOTELS
64,SHALBY,327.00,2.61,-25.02,66.84,25.10,90787.0,-45315.0,135828.0,898.41,30.0,M-SC,11.00,250.0,-0.50,1.01,8.85,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,-2.01,-4.15,26.38,21.14,57485.0,-9429.0,217910.0,-10.87,39.0,X-MC,4.17,74.0,-0.16,1.62,22.83,XY24,NTT,PAINTS
14,CAMPUS,393.0,-1.92,-10.65,49.17,33.28,73707.0,-17872.0,149903.0,-28.06,40.0,M-SC,2.80,222.0,-0.24,1.12,16.19,XY24,NTT,FOOTWEAR
16,CERA,9475.0,-1.63,-31.12,79.86,23.89,96762.0,-54739.0,121164.0,-33.65,30.0,H-SC,9.75,157.0,-0.57,0.90,4.64,OX40N,NTT,CERAMICS
70,SYMPHONY,1306.0,-1.38,-33.48,50.29,-0.03,57250.0,-57302.0,113839.0,-46.65,44.0,M-SC,16.81,206.0,-1.00,0.85,1.35,OX40N,NTT,DURABLES
11,BATAINDIA,2096.0,-1.29,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.68,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.15,-7.28,124.24,107.91,170076.0,-10751.0,136893.0,-24.60,37.0,M-SC,12.38,216.0,-0.06,1.02,2.07,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.19,-13.90,16.17,0.02,24302.0,-24271.0,150288.0,129.35,48.0,M-SC,16.14,234.0,-1.00,1.12,30.72,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.87,-16.68,20.12,0.09,16895.0,-16806.0,83971.0,94.88,43.0,M-SC,2.37,253.0,-0.99,0.62,10.32,OX40N,NTT,DURABLES
66,SIS,528.00,-0.73,-25.43,62.61,21.26,51840.0,-28234.0,82798.0,1951.81,43.0,H-SC,4.61,166.0,-0.54,0.62,11.87,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.65,-25.52,52.35,13.47,105147.0,-68814.0,200853.0,-69.48,30.0,H-SC,4.18,158.0,-0.65,1.49,2.13,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.35,6.18,62.24,72.27,111983.0,10474.0,179922.0,-8.8,66.0,M-LC,1.93,99.0,0.09,1.34,12.74,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.35,6.18,62.24,72.27,111983.0,10474.0,179922.0,-8.80,66.0,M-LC,1.93,99.0,0.09,1.34,12.74,XR,NTT,IT
38,INDIAMART,4810.62,0.31,-3.67,114.61,106.72,136162.0,-4531.0,118805.0,-52.34,32.0,H-SC,1.99,139.0,-0.03,0.88,18.85,AR,ATH,MISC
25,FINCABLES,1641.55,-1.15,-7.28,124.24,107.91,170076.0,-10751.0,136893.0,-24.60,37.0,M-SC,12.38,216.0,-0.06,1.02,2.07,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.09,-2.69,39.38,35.63,80311.0,-5635.0,203939.0,-16.30,37.0,H-MC,3.15,119.0,-0.07,1.52,13.53,AR,ATH,PHARMA
37,IEX,219.00,1.89,-3.56,54.12,48.63,104360.0,-7124.0,192830.0,-35.77,50.0,H-SC,14.05,136.0,-0.07,1.43,7.58,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.46,-21.65,89.48,48.45,140570.0,-43410.0,157096.0,-27.39,25.0,X-MC,12.19,64.0,-0.31,1.17,2.29,X40N,ATH,FINANCE
22,DIXON,18931.72,3.52,-17.89,48.07,21.58,55316.0,-25065.0,115074.0,-58.62,26.0,X-MC,7.13,56.0,-0.45,0.86,3.52,X40N,ATH,IT
75,TMPV,600.00,0.82,-17.96,73.31,42.18,166238.0,-49649.0,226761.0,-26.74,29.0,X-LC,6.05,3.0,-0.30,1.69,0.82,XY24,NTT,AUTO
33,HINDUNILVR,2922.00,0.21,-10.27,26.68,13.67,61540.0,-26403.0,230660.0,-16.94,30.0,X-LC,2.63,14.0,-0.43,1.72,8.89,XY25,NTT,FMCG
23,DMART,5391.45,0.29,-9.05,40.86,28.12,78194.0,-19037.0,191370.0,-21.49,31.0,X-LC,4.94,19.0,-0.24,1.42,12.44,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,1.89,-80.32,534.00,24.79,264720.0,-202283.0,49573.0,-81.32,43.0,X-SC,1.24,86.0,-0.76,0.37,21.19,X40N,NTT,MISC
8,AWL,485.0,-0.02,-21.80,96.28,53.50,227202.0,-65771.0,235980.0,-62.10,33.0,X-MC,1.71,58.0,-0.29,1.76,4.81,XY24,NTT,FMCG
43,ITC,452.0,-0.33,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,45.0,X-LC,1.76,5.0,-0.15,1.46,3.63,X40,NTT,FMCG
36,ICICIPRULI,790.0,-1.03,5.47,24.17,30.97,45055.0,9669.0,186407.0,-19.31,60.0,X-MC,2.02,75.0,0.21,1.39,18.72,X40,ATH,INSURANCE
3,ACC,3906.0,-0.24,-25.28,119.69,64.15,212809.0,-60151.0,177800.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.24,-25.28,119.69,64.15,212809.0,-60151.0,177800.0,-56.31,34.0,X-SC,2.11,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.40,-6.27,42.70,33.76,80376.0,-12585.0,188235.0,-33.85,32.0,X-MC,5.24,57.0,-0.16,1.40,0.00,X40,ATH,FMCG
31,HAVELLS,2069.16,0.30,-10.74,48.42,32.48,141755.0,-35230.0,292761.0,-15.19,36.0,X-MC,5.51,67.0,-0.25,2.18,0.30,X40,ATH,ELECTRICAL
75,TMPV,600.00,0.82,-17.96,73.31,42.18,166238.0,-49649.0,226761.0,-26.74,29.0,X-LC,6.05,3.0,-0.30,1.69,0.82,XY24,NTT,AUTO
53,PAGEIND,51605.07,0.83,-9.61,39.47,26.07,58416.0,-15740.0,148000.0,-33.39,32.0,X-MC,12.14,55.0,-0.27,1.10,0.83,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.09,-29.75,103.33,42.84,47189.0,-19338.0,45668.0,-54.27,41.0,X-SC,36.53,83.0,-0.41,0.34,3.32,XY24,NTT,MISC
15,CAMS,4762.00,1.89,-80.32,534.00,24.79,264720.0,-202283.0,49573.0,-81.32,43.0,X-SC,1.24,86.0,-0.76,0.37,21.19,X40N,NTT,MISC
59,RELAXO,1176.00,-1.09,-47.26,188.80,52.31,144534.0,-68606.0,76554.0,-43.86,44.0,X-SC,6.41,91.0,-0.47,0.57,2.47,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.29,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.68,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR
22,DIXON,18931.72,3.52,-17.89,48.07,21.58,55316.0,-25065.0,115074.0,-58.62,26.0,X-MC,7.13,56.0,-0.45,0.86,3.52,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.22,-13.54,37.96,19.28,113545.0,-46839.0,299117.0,-24.60,58.0,X-LC,5.22,1.0,-0.41,2.23,10.58,X40,ATH,IT
41,INFY,2275.00,0.35,8.27,43.05,54.87,145834.0,25867.0,338755.0,-15.65,61.0,X-LC,3.22,2.0,0.18,2.52,17.23,X40,BTT,IT
75,TMPV,600.00,0.82,-17.96,73.31,42.18,166238.0,-49649.0,226761.0,-26.74,29.0,X-LC,6.05,3.0,-0.30,1.69,0.82,XY24,NTT,AUTO
81,VBL,671.64,0.76,-3.74,40.94,35.67,124474.0,-11803.0,304039.0,-15.14,56.0,X-LC,4.59,4.0,-0.09,2.26,9.60,X40N,ATH,FMCG
43,ITC,452.00,-0.33,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,45.0,X-LC,1.76,5.0,-0.15,1.46,3.63,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.52,-18.21,122.11,81.67,93695.0,-17080.0,76730.0,6894.44,39.0,L-SC,13.82,271.0,-0.18,0.57,48.59,XR,NTT,CERAMICS
50,MASFIN,398.61,0.13,-2.10,24.66,22.05,23655.0,-2055.0,95925.0,-15.54,56.0,H-SC,7.10,164.0,-0.09,0.71,39.43,XR,ATH,FINANCE
13,BSOFT,831.70,1.63,-22.04,93.42,50.79,102435.0,-31003.0,109650.0,0.12,67.0,H-SC,5.87,151.0,-0.30,0.82,27.67,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.19,-13.90,16.17,0.02,24302.0,-24271.0,150288.0,129.35,48.0,M-SC,16.14,234.0,-1.00,1.12,30.72,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.64,-2.60,24.18,20.95,59292.0,-6556.0,245212.0,-3.58,47.0,X-LC,18.67,31.0,-0.11,1.82,32.10,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.63,-22.04,93.42,50.79,102435.0,-31003.0,109650.0,0.12,67.0,H-SC,5.87,151.0,-0.30,0.82,27.67,XR,ATH,IT
84,WIPRO,420.00,0.35,6.18,62.24,72.27,111983.0,10474.0,179922.0,-8.80,66.0,M-LC,1.93,99.0,0.09,1.34,12.74,XR,NTT,IT
56,RAJESHEXPO,518.00,0.48,-63.07,170.99,0.07,86615.0,-86522.0,50655.0,1756.06,56.0,L-SC,2.37,268.0,-1.00,0.38,25.98,OX40N,NTT,JEWELLERY
32,HCLTECH,1922.01,0.05,8.18,15.30,24.73,40041.0,19779.0,261703.0,12.76,63.0,X-LC,6.20,7.0,0.49,1.95,24.66,X40,ATH,IT
50,MASFIN,398.61,0.13,-2.10,24.66,22.05,23655.0,-2055.0,95925.0,-15.54,56.0,H-SC,7.10,164.0,-0.09,0.71,39.43,XR,ATH,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.77
1,25,45.76
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.09
MC    30.25
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.33
X40      22.41
X40N     10.83
XR       10.14
XY25      9.40
AR        9.37
OX40N     7.77
SR        1.02
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.56
X-MC    23.49
X-LC    21.09
M-SC    12.15
X-SC     6.95
H-MC     5.01
L-SC     1.43
M-MC     1.40
M-LC     1.34
H-LC     1.18
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-7.64,42.45
IT,12.77,-17.62,81.21
FINANCE,9.58,-20.42,71.30
ELECTRICAL,6.24,-12.13,53.20
MISC,6.12,-57.91,122.60
PAINTS,6.05,-13.39,30.24
INSURANCE,4.63,-1.79,37.66
PHARMA,4.11,-1.06,33.41
AUTO,3.43,-24.72,78.19


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3228905.0,21
AR,1313390.0,10
XR,1298720.0,13
X40,1026000.0,14
X40N,992892.0,9
OX40N,749387.0,10
XY25,377781.0,6
SR,283493.0,2
MH,84626.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3677413.0,25
M-SC,1426810.0,15
X-MC,1324499.0,16
X-SC,985505.0,8
X-LC,929134.0,11
H-MC,406073.0,3
L-SC,269401.0,3
M-LC,111983.0,1
H-LC,75081.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1285289.0      6
           AR         901334.0      5
M-SC       XY24       811173.0      6
H-SC       XR         774584.0      7
X-SC       X40N       507567.0      3
X-MC       X40        459687.0      7
           XY24       397681.0      3
X-LC       X40        383252.0      5
H-SC       OX40N      348087.0      4
M-SC       OX40N      314685.0      5
X-SC       XY24       294877.0      3
H-SC       SR         283493.0      2
X-MC       X40N       282657.0      4
X-LC       XY24       244276.0      2
H-MC       AR         210464.0      2
X-LC       X40N       202668.0      2
H-MC       XY24       195609.0      1
X-MC       XY25       184474.0      2
X-SC       X40        183061.0      2
L-SC       XR         182786.0      2
M-SC       XR         174441.0      2
           AR         126511.0      2
M-LC       XR         111983.0      1
X-LC       XY25        98938.0      2
L-SC       OX40N       86615.0      1
H-SC       MH          84626.0      1
H-LC       AR          75081.0      1
M-MC       XY25        57172.0      1
L-MC       XR          54926.0      1
L-LC       XY25        37197.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
